In [ ]:
! pip install rouge


In [ ]:
from rouge import Rouge
import pandas as pd

# Load your data
beam_summaries_path = '/content/drive/MyDrive/BugSum-master/excel sheet/beam_bug_report_summaries.xlsx'
bart_summaries_path = '/content/drive/MyDrive/BugSum-master/excel sheet/bart_bug_report_summaries.xlsx'

beam_df = pd.read_excel(beam_summaries_path)
bart_df = pd.read_excel(bart_summaries_path)

# Initialize ROUGE scorer
rouge = Rouge()

# Prepare data for ROUGE calculation
beam_summaries_dict = beam_df.set_index('BugNum')['Summary'].to_dict()
bart_summaries_dict = bart_df.set_index('BugNum')['Refined_Summary'].to_dict()

# Calculate ROUGE scores for each BART summary against the corresponding Beam summary as reference
scores = []
for bug_num, bart_summary in bart_summaries_dict.items():
    if bug_num in beam_summaries_dict:
        score = rouge.get_scores(bart_summary, beam_summaries_dict[bug_num], avg=True)
        scores.append(score)

# Aggregate average ROUGE scores
average_scores = {'rouge-1': {'f': 0, 'p': 0, 'r': 0}, 'rouge-2': {'f': 0, 'p': 0, 'r': 0}, 'rouge-l': {'f': 0, 'p': 0, 'r': 0}}

for score in scores:
    for metric, values in score.items():
        average_scores[metric]['f'] += values['f']
        average_scores[metric]['p'] += values['p']
        average_scores[metric]['r'] += values['r']

num_scores = len(scores)
for metric in average_scores:
    average_scores[metric]['f'] /= num_scores
    average_scores[metric]['p'] /= num_scores
    average_scores[metric]['r'] /= num_scores

print("Average ROUGE scores:", average_scores)




Average ROUGE scores: {'rouge-1': {'f': 0.571995900881209, 'p': 0.6296426126954119, 'r': 0.5543852774138259}, 'rouge-2': {'f': 0.47795516153274625, 'p': 0.5394648758067383, 'r': 0.4586407406265393}, 'rouge-l': {'f': 0.5631154151487665, 'p': 0.6200688971989738, 'r': 0.5454635157549161}}
